In [ ]:
from utils import *
import pickle
import datetime
import scipy
from scipy.optimize import minimize

In [ ]:
params=Parameters(B=0., u=100., Tl=1., Tr=1., gamma=0.25, kerntype='RTD', dband=1e4, countingleads=[0,2])

In [ ]:
def power(Vs,params):
    vg,vb = Vs
    system = anderson(vg,vb,params)
    system.solve()
    return -vb*np.sum(system.current[[0,2]])

In [ ]:

if __name__ == '__main__':
    Pmax = []
    params=Parameters(B=0., u=100., Tl=1., Tr=1., gamma=.25, kerntype='RTD', dband=1e4, countingleads=[0,2])
    for dT in tqdm(np.linspace(0.05,3,50), desc='dT'):
        params.Tl = 1+dT
        res = dT, minimize( lambda x: -power(x,params), [-1,-1],tol=1e-8, options={'maxiter':200})
        Pmax.append(res)
        #print(dT, res[1].x, -res[1].fun)

In [ ]:

if __name__ == '__main__':
    Pmax_pauli = []
    params=Parameters(B=0., u=100., Tl=1., Tr=1., gamma=.25, kerntype='Pauli', dband=1e4, countingleads=[0,2])
    for dT in tqdm(np.linspace(0.05,3,50), desc='dT'):
        params.Tl = 1+dT
        res = dT, minimize( lambda x: -power(x,params), [-1,-1],tol=1e-8, options={'maxiter':200})
        Pmax_pauli.append(res)
        #print(dT, res[1].x, -res[1].fun)

In [ ]:

if __name__ == '__main__':
    Pmax_pauli_u0 = []
    params=Parameters(B=0., u=0., Tl=1., Tr=1., gamma=.25, kerntype='Pauli', dband=1e4, countingleads=[0,2])
    for dT in tqdm(np.linspace(0.05,3,50), desc='dT'):
        params.Tl = 1+dT
        res = dT, minimize( lambda x: -power(x,params), [-1,-1],tol=1e-8, options={'maxiter':200})
        Pmax_pauli_u0.append(res)
        #print(dT, res[1].x, -res[1].fun)

In [ ]:

if __name__ == '__main__':
    Pmax_u0 = []
    params=Parameters(B=0., u=0., Tl=1., Tr=1., gamma=.25, kerntype='RTD', dband=1e4, countingleads=[0,2])
    for dT in tqdm(np.linspace(0.05,3,50), desc='dT'):
        params.Tl = 1+dT
        res = dT, minimize( lambda x: -power(x,params), [-1,-1],tol=1e-8, options={'maxiter':200})
        Pmax_u0.append(res)
        #print(dT, res[1].x, -res[1].fun)

In [ ]:
if __name__ == '__main__':
    Pmax_lb = []
    for dT in tqdm(np.linspace(0.05,3,50), desc='dT'):
        res = dT, minimize( lambda x: x[1]*SRL_exact(.25,x[0],x[1],1.+dT,1.,dband=1e2)[0], [-1,-1],tol=1e-8, options={'maxiter':200})
        Pmax_lb.append(res)

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# pickle Pmax
with open(timestamp+'Pmax_outside_coulomb.pkl', 'wb') as f:
    pickle.dump(Pmax, f)
with open(timestamp+'Pmax_pauli_outside_coulomb.pkl', 'wb') as f:
    pickle.dump(Pmax_pauli, f)
with open(timestamp+'Pmax_u0_outside_coulomb.pkl', 'wb') as f:
    pickle.dump(Pmax_u0, f)
with open(timestamp+'Pmax_lb_outside_coulomb.pkl', 'wb') as f:
    pickle.dump(Pmax_lb, f)
with open(timestamp+'Pmax_pauli_u0_outside_coulomb.pkl', 'wb') as f:
    pickle.dump(Pmax_pauli_u0, f)


In [ ]:
#timestamp = '20240902142354'
files = [timestamp+'Pmax_outside_coulomb.pkl', timestamp+'Pmax_pauli_outside_coulomb.pkl', timestamp+'Pmax_u0_outside_coulomb.pkl', timestamp+'Pmax_lb_outside_coulomb.pkl', timestamp+'Pmax_pauli_u0_outside_coulomb.pkl']
# import data from files
#with open(files[0], 'rb') as f:
#    Pmax_inside_coulomb = pickle.load(f)
with open(files[0], 'rb') as f:
    Pmax_outside_coulomb = pickle.load(f)
with open(files[1], 'rb') as f:
    Pmax_pauli_outside_coulomb = pickle.load(f)
with open(files[2], 'rb') as f:
    Pmax_u0_outside_coulomb = pickle.load(f)
with open(files[3], 'rb') as f:
    Pmax_lb_outside_coulomb = pickle.load(f)
with open(files[4], 'rb') as f:
    Pmax_pauli_u0_outside_coulomb = pickle.load(f)

In [ ]:
dt=np.array([i[0] for i in Pmax_outside_coulomb])
Vg_rtd=np.array([i[1].x[0] for i in Pmax_outside_coulomb])
Vg_pauli=np.array([i[1].x[0] for i in Pmax_pauli_outside_coulomb])
Vg_pauli_u0=np.array([i[1].x[0] for i in Pmax_pauli_u0_outside_coulomb])
Vg_u0=np.array([i[1].x[0] for i in Pmax_u0_outside_coulomb])
Vg_lb=np.array([i[1].x[0] for i in Pmax_lb_outside_coulomb])
Vb_rtd=np.array([i[1].x[1] for i in Pmax_outside_coulomb])
Vb_pauli=np.array([i[1].x[1] for i in Pmax_pauli_outside_coulomb])
Vb_pauli_u0=np.array([i[1].x[1] for i in Pmax_pauli_u0_outside_coulomb])
Vb_u0=np.array([i[1].x[1] for i in Pmax_u0_outside_coulomb])
Vb_lb=np.array([i[1].x[1] for i in Pmax_lb_outside_coulomb])
params_rtd=[Parameters(B=0., u=100., Tl=1.+dT, Tr=1., gamma=.25, kerntype='pyRTDnoise', dband=1e4, countingleads=[0,2]) for dT in dt]
params_pauli=[Parameters(B=0., u=100., Tl=1.+dT, Tr=1., gamma=.25, kerntype='pyPauli', dband=1e4, countingleads=[0,2]) for dT in dt]
params_rtd_u0=[Parameters(B=0., u=0., Tl=1.+dT, Tr=1., gamma=.25, kerntype='pyRTDnoise', dband=1e4, countingleads=[0,2]) for dT in dt]
params_pauli_u0=[Parameters(B=0., u=0., Tl=1.+dT, Tr=1., gamma=.25, kerntype='pyPauli', dband=1e4, countingleads=[0,2]) for dT in dt]
Vb=[Vb_rtd,Vb_pauli,Vb_u0,Vb_pauli_u0,Vb_lb]

In [ ]:
def main(Vg, Vb, params):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(solve_bias_gate_p,Vg,Vb,params),total=len(Vg))),dtype=object)
    return results

def main_srl(Vg,Vb,dt):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(SRL_exact,[.25]*len(Vg),Vg,Vb,dt+1.,[1.]*len(Vg)),total=len(Vg))))
    return results

In [ ]:
results = main(Vg_rtd, Vb_rtd, params_rtd)

In [ ]:
results_pauli = main(Vg_pauli, Vb_pauli, params_pauli)

In [ ]:
results_rts_u0 = main(Vg_u0, Vb_u0, params_rtd_u0)

In [ ]:
results_srl = main_srl(Vg_lb,Vb_lb,dt)

In [ ]:
results_pauli_u0 = main(Vg_pauli_u0, Vb_pauli_u0, params_pauli_u0)

# TUR
$2 \leq \frac{S}{I}\frac{V}{T_c}\frac{n_c-n}{n}$

for noise:
$ S \geq 2 I \frac{T_c}{V} \frac{n}{n_c-n}$

for current:

$I \leq \frac{S}{2}\frac{V}{T_c}\frac{n_c-n}{n}$ if $I>0$

$I \geq \frac{S}{2}\frac{V}{T_c}\frac{n_c-n}{n}$ if $I<0$

for efficiency:
$ n \leq \frac{n_c}{\frac{2IT_c}{SV}+1}$

In [ ]:

I,S,Q,P,n,TUR,TURS,TURI,TURn = [np.zeros((5,len(dt))) for i in range(9)]


In [ ]:
for i,res in enumerate([results, results_pauli, results_rts_u0, results_pauli_u0]):
    I[i] = np.array([r[0][0] for r in res]).real
    S[i] = np.array([r[0][1] for r in res]).real
    Q[i] = np.array([r[2][0]+r[2][2] for r in res]).real
    P[i] = - I[i]*Vb[i]
    nc=1-1/(1+dt)
    n[i] = P[i]/Q[i]
    TUR[i]=-S[i]/I[i]*Vb[i]/1*(nc-n[i])/n[i]
    TURS[i]=-2*I[i]/Vb[i]*1/(nc-n[i])*n[i]
    TURI[i]=-S[i]/2*Vb[i]/1*(nc-n[i])/n[i]
    TURn[i]=nc/((-2*I[i]*1)/S[i]/Vb[i]+1)

In [ ]:
I[4]= 2*np.array([res[0] for res in results_srl])
S[4] = 2*np.array([res[1] for res in results_srl])
Q[4] = 2*np.array([res[2] for res in results_srl])
P[4] = -I[4]*Vb[4]
n[4] = P[4]/Q[4]
TUR[4] = -S[4]/I[4]*Vb[4]/1*(nc-n[4])/n[4]
TURS[4] = -2*I[4]/Vb[4]*1/(nc-n[4])*n[4]
TURI[4] = -S[4]/2*Vb[4]/1*(nc-n[4])/n[4]
TURn[4] = nc/((-2*I[4]*1)/S[4]/Vb[4]+1)

In [ ]:
purples = ['#f7fcfd','#e0ecf4','#bfd3e6','#9ebcda','#8c96c6','#8c6bb1','#88419d','#810f7c','#4d004b']
oranges = ['#ffffe5','#fff7bc','#fee391','#fec44f','#fe9929','#ec7014','#cc4c02','#993404','#662506']


In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
cols=[oranges[5],purples[5],oranges[3],purples[3],'k']
labels=['Second order U=100','First order U=100','Second order U=0', 'First order U=0', 'Exact LB U=0']
gamma=0.25
lss=['-','-','-','-','--'] 
# set up figure with 6 subplots
fig, ax = plt.subplots(2, 3, figsize=(12,6), sharex=True,constrained_layout=True)
for i in range(5):
    # in first panel plot dt vs TURI and dt vs I
    curr = ax[0,0].plot(dt, I[i]/gamma, label='I'+' '+labels[i],color=cols[i], linestyle=lss[i])
    #ax[0,0].plot(dt, TURI[i],color=curr[0].get_color(), linestyle=':' , label='_no_legend_')
    # fill area under TURI curve to axis in grey
    #ax[0,0].fill_between(dt, TURI[i], TURI.max(),color=curr[0].get_color(), alpha=0.15)

    # in second panel plot dt vs TURS and dt vs S
    noi = ax[0,1].plot(dt, S[i]/gamma, label='_no_legend_',color=cols[i], linestyle=lss[i])
    ax[0,1].plot(dt, TURS[i]/gamma, color=noi[0].get_color(), linestyle=':', label='_no_legend_')
    # fill area under TURS curve to axis in grey
    ax[0,1].fill_between(dt, TURS[i]/gamma, TURS.min(), color='lightgray', alpha=0.20)

    fano = ax[0,2].plot(dt, S[i]/I[i], label='_no_legend_',color=cols[i], linestyle=lss[i])

    pow = ax[1,0].plot(dt, P[i]/gamma, label='_no_legend_',color=cols[i], linestyle=lss[i])
    
    heat = ax[1,1].plot(dt, Q[i]/gamma, label='_no_legend_',color=cols[i], linestyle=lss[i])

    # in third panel plot dt vs TURn and dt vs n
    eff = ax[1,2].plot(dt, n[i]/nc, label='_no_legend_',color=cols[i], linestyle=lss[i])
    ax[1,2].plot(dt, TURn[i]/nc, color=eff[0].get_color(),linestyle=':', label='_no_legend_')
    # fill area over TURn curve to axis in grey
    ax[1,2].fill_between(dt, TURn[i]/nc, 1, color='lightgray', alpha=0.20)

ax[0,0].set_ylim(0,.11)
ax[0,1].set_ylim(0.05,.185)
ax[0,2].set_ylim(0,5)
ax[1,2].set_ylim(0.3,0.75)

# ax[0,0].plot(dt, TUR_lb,':', label='TUR SRL',color='k')
# ax[0,1].plot(dt, S_lb,':', label='S SRL',color='k')
# #ax[0,1].plot(dt, TURS_lb, label='TURS SRL',color='k')
# ax[1,0].plot(dt, I_lb,':', label='I SRL',color='k')
# #ax[1,0].plot(dt, TURI_lb, label='TURI SRL',color='k')
# ax[1,1].plot(dt, n_lb/nc,':', label='n SRL',color='k')
# #ax[1,1].plot(dt, TURn_lb, label='TURn SRL',color='k')

titles = [r'Current $I/\Gamma$',r'Noise $S/\Gamma$',r'    Fano factor $F=S/I$',r'Power $P/(\Gamma \cdot T)$',r'Heat current $Q/\Gamma$',r'Efficiency $n/n_c$']
for i,a in enumerate(ax.flatten()):
    a.set_xmargin(0)
    a.set_ymargin(0)
    a.set_xlim(0,3)
    # label and title as text in top left of plot
    a.text(0.02,0.98,f'({chr(97+i)}) '+titles[i],fontsize='large', horizontalalignment='left',verticalalignment='top',transform=a.transAxes)
    a.grid(True,which='both',axis='both',linestyle='--',linewidth=0.5)
for a in ax[1,:]:
    a.set_xlabel(r'$\Delta T/T$')#('Temperature difference $\Delta T$')

fig.subplots_adjust(hspace=0.05)

lab=[labels[0],labels[1],labels[2],labels[3],labels[4],'TUR bounds']

ax[0,0].plot([],[],':',color='gray')

plt.figlegend(loc='upper center', ncol=3, labels=lab)#, bbox_to_anchor=(0.5,0.02))

plt.savefig('fig4.pdf',bbox_inches='tight')

plt.show()



In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
plt.plot(Vg_rtd,Vb_rtd,color=oranges[5])
plt.plot(Vg_u0,Vb_u0,color=oranges[3])
plt.plot(Vg_pauli,Vb_pauli,color=purples[5])
plt.plot(Vg_pauli_u0,Vb_pauli_u0,color=purples[3])
plt.plot(Vg_lb,Vb_lb,'k--')
plt.legend(['Second order U=100', 'Second order U=0', 'First order U=100', 'First order U=0', 'Exact LB U=0'])
plt.xlabel('$V_g/T$')
plt.ylabel('$V_b/T$')
plt.margins(0)
plt.savefig('maxppdt.pdf',bbox_inches='tight')